In [1]:
import pandas as pd
import os
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [2]:
os.chdir('C:/Users/wissam_T/Desktop/5th/2nd semester/dm/h.w 1/project')

In [3]:
df_merged = pd.read_csv('datasets/Merged_trips_with_stations.csv')

In [28]:
df_merged.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'duration_min', 'geometry_start',
       'geometry_end', 'Zone_start_zone_name', 'Zone_start_GIS_ID',
       'Zone_start_OBJECTID', 'Zone_start_geometry', 'Zone_end_zone_name',
       'Zone_end_GIS_ID', 'Zone_end_OBJECTID', 'Zone_end_geometry', 'date',
       'Weather_temp', 'Weather_windspeedmean', 'Weather_sunrise',
       'Weather_sunset', 'Weather_conditions', 'Bikeshare_NAME_start',
       'Bikeshare_STATION_TYPE_start', 'Bikeshare_CAPACITY_start',
       'Bikeshare_REGION_ID_start', 'Bikeshare_REGION_NAME_start',
       'Bikeshare_geometry_start', 'Bikeshare_NAME_end',
       'Bikeshare_STATION_TYPE_end', 'Bikeshare_CAPACITY_end',
       'Bikeshare_REGION_ID_end', 'Bikeshare_REGION_NAME_end',
       'Bikeshare_geometry_end', 'StartStation_NAME', 'EndStation_NAME'],
      dtype='object')

In [29]:
print(df_merged['Bikeshare_CAPACITY_start'].isna().sum())
print(df_merged['Bikeshare_CAPACITY_end'].isna().sum())

0
28


In [30]:
df_merged.shape

(97, 40)

In [43]:
df_merged_copy = df_merged.copy()

In [41]:
start_stations = df_merged[['start_station_id', 'Bikeshare_CAPACITY_start']].rename(
    columns={'start_station_id': 'station_id', 'Bikeshare_CAPACITY_start': 'capacity'}
)

end_stations = df_merged[['end_station_id', 'Bikeshare_CAPACITY_end']].rename(
    columns={'end_station_id': 'station_id', 'Bikeshare_CAPACITY_end': 'capacity'}
)

# Combine both into one list of stations
stations = pd.concat([start_stations, end_stations])

# Remove nulls and duplicates (in case some stations appear twice)
stations = stations.drop_duplicates(subset='station_id')

# Step 2: Calculate thresholds (33rd and 66th percentiles)
low_threshold = np.percentile(stations['capacity'], 33)
high_threshold = np.percentile(stations['capacity'], 66)

# Step 3: Define a function to assign category
def assign_size_category(cap):
    if cap < low_threshold:
        return 'Small'
    elif cap < high_threshold:
        return 'Medium'
    else:
        return 'Large'

# Step 4: Apply the function to create the new column
stations['station_size'] = stations['capacity'].apply(assign_size_category)

# Now `stations` has: station_id, capacity, and station_size
print(stations.head())


    station_id  capacity station_size
0      31519.0      15.0        Large
7      31651.0      19.0        Large
9      31939.0      17.0        Large
10     30200.0      27.0        Large
11     32265.0      12.0        Large


In [49]:
# Merge back to the original df for start stations
df = df_merged_copy.merge(
    stations[['station_id', 'station_size']],
    left_on='start_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'start_station_size'}).drop(columns=['station_id'])

# Merge back again for end stations
df = df.merge(
    stations[['station_id', 'station_size']],
    left_on='end_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'end_station_size'}).drop(columns=['station_id'])


In [51]:
df.shape

(97, 42)

In [52]:
df_merged_copy.to_csv('added_size.csv', index=False)